# Preentrenamiento del modelo especializado en matrículas.

In [1]:
from ultralytics import YOLO

model = YOLO('yolo11n.pt')

results = model.train(
  data="data.yaml",
  epochs=100,
  imgsz=640,
  batch=4,
  patience=50
)

Ultralytics 8.3.222  Python-3.9.5 torch-2.8.0+cpu CPU (Intel Core i5-10400F 2.90GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

### Carga del modelo entrenado

In [3]:
from ultralytics import YOLO 
import cv2

# --- 1. Configuración ---
vehicle_model = YOLO('yolo11n.pt')
plate_model = YOLO('runs/detect/train3/weights/best.pt')

vid_route = "./20251023_143820.mp4"
vid_out_path = "./"
# Clases a detectar (0: persona, 2: coche, 5: bus, 7: camión)
classes_to_detect = [0, 2, 5, 7] 

cap = cv2.VideoCapture(vid_route)
if not cap.isOpened():
    print(f"Error: No se pudo abrir el video '{vid_route}'")
    exit()

# --- 2. Definir Tamaño de Visualización ---
# Obtenemos las dimensiones originales del vídeo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Establecemos un ANCHO máximo para la ventana (ej. 1280 píxeles)
DISPLAY_WIDTH = 1280 
# Calculamos el alto manteniendo la proporción
aspect_ratio = frame_height / frame_width
DISPLAY_HEIGHT = int(DISPLAY_WIDTH * aspect_ratio)

print(f"Video Original: {frame_width}x{frame_height} -> Mostrando a: {DISPLAY_WIDTH}x{DISPLAY_HEIGHT}")

# --- 3. Bucle de Procesamiento ---
while True: 
    ret, frame = cap.read()
    if not ret: 
        print("THE END")
        break
    
    # --- Detección de Vehículos y Personas ---
    resultados = vehicle_model(
      frame,
      classes=classes_to_detect,
      imgsz=640,
      conf=0.4,
      verbose=False
    )

    if resultados[0].boxes is not None:
        for box in resultados[0].boxes:
            # Coordenadas del objeto
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Clase y confianza
            cls = int(box.cls[0])
            # Corrección: Obtenemos el nombre de la clase desde el modelo
            currentClass = vehicle_model.names[cls] 
            confidence = box.conf[0]

            # Dibujar caja para el objeto general
            color = (255, 0, 0) # Azul por defecto
            if currentClass == "person":
                color = (0, 0, 255) # Rojo para personas

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f'{currentClass} {confidence:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # --- Detección de Matrículas (sólo si es vehículo) ---
            if currentClass in ["car", "bus", "truck"]:
                
                # Recortar la imagen del vehículo
                img_vehiculo = frame[y1:y2, x1:x2]

                if img_vehiculo.size == 0:
                    continue

                # Ejecutar Modelo Especialista de matrículas
                results_placa = plate_model(
                    img_vehiculo, 
                    conf=0.5, # Confianza más alta para matrículas
                    imgsz=640,
                    verbose=False
                )

                if results_placa[0].boxes is not None:
                    # Dibujar las matrículas encontradas
                    for box_placa in results_placa[0].boxes:
                        # Coordenadas RELATIVAS al recorte
                        px1, py1, px2, py2 = map(int, box_placa.xyxy[0])
                        placa_conf = box_placa.conf[0]
                        
                        # Convertir a coordenadas ABSOLUTAS (del frame original)
                        abs_px1 = px1 + x1
                        abs_py1 = py1 + y1
                        abs_px2 = px2 + x1
                        abs_py2 = py2 + y1
                        
                        # Dibuja la matrícula
                        color_placa = (0, 255, 0) # Verde
                        cv2.rectangle(frame, (abs_px1, abs_py1), (abs_px2, abs_py2), color_placa, 2)
                        cv2.putText(frame, f"Matricula {placa_conf:.2f}", (abs_px1, abs_py1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_placa, 2)
                        # (Línea duplicada eliminada)

    # --- 4. Redimensionar y Mostrar ---
    # Redimensionamos el frame (con todas las anotaciones) al tamaño deseado
    display_frame = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT), interpolation=cv2.INTER_AREA)
    
    cv2.imshow("Prototipo Final (Pipeline de 2 Modelos)", display_frame)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- 5. Limpieza ---
cap.release()
cv2.destroyAllWindows()
print("Procesamiento detenido por el usuario.")

Video Original: 1920x1080 -> Mostrando a: 1280x720
THE END
Procesamiento detenido por el usuario.


In [ ]:
from ultralytics import YOLO 
import cv2

 # --- 1. Configuración ---
vehicle_model = YOLO('yolo11n.pt')
plate_model = YOLO('runs/detect/train3/weights/best.pt')

vid_route = "./20251023_143820.mp4"
vid_out_path = "./"
# Clases a detectar (0: persona, 2: coche, 5: bus, 7: camión)
classes_to_detect = [0, 2, 5, 7] 

frame = cv2.imread("Opel-Astra.png")
if frame is None:
  print("No se pudo detectar la imagen")
  exit()

resultados = vehicle_model(
  frame,
  classes=classes_to_detect,
  conf=0.4,
  verbose=False
)

detection_count = 0

if resultados[0].boxes is not None:
    for box in resultados[0].boxes:
        detection_count += 1
        # Coordenadas del vehículo
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        # Clase y confianza
        cls = int(box.cls[0])
        currentClass = vehicle_model.names[cls] 
        confidence = box.conf[0]

        # Dibujar caja para el vehículo
        color_vehiculo = (255, 0, 0) # Azul
        cv2.rectangle(frame, (x1, y1), (x2, y2), color_vehiculo, 2)
        label = f'{currentClass} {confidence:.2f}'
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_vehiculo, 2)

        # --- 4. Detección de Matrículas (Modelo Especialista) ---
        
        # Recortar la imagen del vehículo (Región de Interés o ROI)
        img_vehiculo = frame[y1:y2, x1:x2]

        if img_vehiculo.size == 0:
            continue

        # Ejecutar Modelo Especialista de matrículas
        results_placa = plate_model(
            img_vehiculo, 
            conf=0.5, # Usamos una confianza más alta para matrículas
            verbose=False
        )

        if results_placa[0].boxes is not None:
            # Dibujar las matrículas encontradas
            for box_placa in results_placa[0].boxes:
                # Coordenadas RELATIVAS al recorte (ROI)
                px1, py1, px2, py2 = map(int, box_placa.xyxy[0])
                placa_conf = box_placa.conf[0]
                
                # Convertir a coordenadas ABSOLUTAS (del frame original)
                abs_px1 = px1 + x1
                abs_py1 = py1 + y1
                abs_px2 = px2 + x1
                abs_py2 = py2 + y1
                
                # Dibuja la matrícula en el frame original
                color_placa = (0, 255, 0) # Verde
                cv2.rectangle(frame, (abs_px1, abs_py1), (abs_px2, abs_py2), color_placa, 2)
                cv2.putText(frame, f"Matricula {placa_conf:.2f}", (abs_px1, abs_py1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_placa, 2)


frame_height, frame_width = frame.shape[:2]
DISPLAY_WIDTH = 900

if frame_width > DISPLAY_WIDTH:
    aspect_ratio = frame_height / frame_width
    DISPLAY_HEIGHT = int(DISPLAY_WIDTH * aspect_ratio)
    display_frame = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT), interpolation=cv2.INTER_AREA)
else:
    display_frame = frame

cv2.imshow("Deteccion en Imagen (Vehiculos y Matriculas)", display_frame)
print("Presiona cualquier tecla para cerrar la ventana.")
cv2.waitKey(0) # Espera indefinidamente hasta que se presione una tecla
cv2.destroyAllWindows()
print("Ventana cerrada.")

Presiona cualquier tecla para cerrar la ventana.
Ventana cerrada.
